# 예제 3.06-08: 텐서 장치 변환 및 NumPy 연동

## 학습목표
1. **텐서의 장치 간 변환** 방법 익히기 - CPU ↔ GPU (CUDA/MPS)
2. **NumPy 배열과 텐서 간 변환** 이해하기
3. **GPU 텐서를 NumPy로 변환**하는 올바른 방법 학습하기

---

#### 예제 3.06 텐서 장치 변환 (CUDA)

**장치 변환 메서드**
- `.cuda()`: CPU → GPU(CUDA)로 이동
- `.cpu()`: GPU → CPU로 이동
- `.to(device)`: 지정한 장치로 이동 (권장 방법)

In [ ]:
import torch

# CPU에 텐서 생성
cpu = torch.FloatTensor([1, 2, 3])

# .cuda(): CPU 텐서를 GPU로 이동
gpu = cpu.cuda()

# .cpu(): GPU 텐서를 다시 CPU로 이동
gpu2cpu = gpu.cpu()

# .to('device'): macos는mps, windows는cuda이기 때문에 감지하는 장치에 따라 사용하려면 변수지정하여 사용 가능
# .to("cuda"): 권장하는 장치 변환 방법
cpu2gpu = cpu.to("cuda")


print(cpu)      # CPU 텐서
print(gpu)      # GPU 텐서 (device='cuda:0')
print(gpu2cpu)  # 다시 CPU로 이동한 텐서
print(cpu2gpu)  # to()로 GPU로 이동한 텐서

tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')
tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')


---

#### 예제 3.06 텐서 장치 변환 (Apple Silicon: MPS)

**Apple Silicon에서의 GPU 사용**
- `.to("mps")`: CPU → MPS(Apple GPU)로 이동
- `.cpu()`: MPS → CPU로 이동

In [2]:
# 애플 실리콘이 탑재된 맥 사용자의 경우
import torch

# CPU에 텐서 생성
cpu = torch.FloatTensor([1, 2, 3])

# .to("mps"): CPU 텐서를 MPS(Apple GPU)로 이동
gpu = cpu.to("mps")

# .cpu(): MPS 텐서를 다시 CPU로 이동
gpu2cpu = gpu.cpu()

print(cpu)      # CPU 텐서
print(gpu)      # MPS 텐서 (device='mps:0')
print(gpu2cpu)  # 다시 CPU로 이동한 텐서

RuntimeError: PyTorch is not linked with support for mps devices

---

#### 예제 3.07 넘파이 배열의 텐서 변환

**NumPy → PyTorch 텐서 변환 방법**
- `torch.tensor(ndarray)`: 새로운 메모리에 복사 (권장)
- `torch.Tensor(ndarray)`: FloatTensor로 변환
- `torch.from_numpy(ndarray)`: 메모리 공유 (원본 변경 시 텐서도 변경됨)

In [3]:
import torch
import numpy as np

# NumPy 배열 생성 (uint8 타입)
ndarray = np.array([1, 2, 3], dtype=np.uint8)

# torch.tensor(): 새 메모리에 복사, 원본 dtype 유지
print(torch.tensor(ndarray))  # torch.uint8

# torch.Tensor(): FloatTensor로 변환 (dtype 변경됨)
print(torch.Tensor(ndarray))  # torch.float32

# torch.from_numpy(): 메모리 공유 (주의: 원본 수정 시 텐서도 변경)
print(torch.from_numpy(ndarray))  # torch.uint8, 메모리 공유

tensor([1, 2, 3], dtype=torch.uint8)
tensor([1., 2., 3.])
tensor([1, 2, 3], dtype=torch.uint8)


---

#### 예제 3.08 텐서의 넘파이 배열 변환

**PyTorch 텐서 → NumPy 변환**
- GPU 텐서는 직접 NumPy로 변환 불가
- 반드시 `.cpu()`로 CPU로 이동 후 `.numpy()` 호출
- `.detach()`: 연산 그래프에서 분리 (기울기 계산 중단)

In [4]:
# CUDA GPU 텐서 → NumPy 변환
import torch

# GPU에 텐서 생성
tensor = torch.cuda.FloatTensor([1, 2, 3])

# GPU 텐서를 NumPy로 변환하는 올바른 방법:
# 1. detach(): 연산 그래프에서 분리
# 2. cpu(): GPU → CPU로 이동
# 3. numpy(): NumPy 배열로 변환
ndarray = tensor.detach().cpu().numpy()

print(ndarray)        # [1. 2. 3.]
print(type(ndarray))  # <class 'numpy.ndarray'>

[1. 2. 3.]
<class 'numpy.ndarray'>


C:\Users\ysj58\AppData\Local\Temp\ipykernel_3288\3444665124.py:5: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:80.)
  tensor = torch.cuda.FloatTensor([1, 2, 3])


In [5]:
# MPS(Apple Silicon) 텐서 → NumPy 변환
import torch

# MPS에 텐서 생성
tensor = torch.FloatTensor([1, 2, 3]).to("mps")

# MPS 텐서를 NumPy로 변환 (동일한 방법)
ndarray = tensor.detach().cpu().numpy()

print(ndarray)        # [1. 2. 3.]
print(type(ndarray))  # <class 'numpy.ndarray'>

RuntimeError: PyTorch is not linked with support for mps devices